<a href="https://colab.research.google.com/github/ZhiyaoShu/Mchine-learning-models/blob/main/tweet_sentiment_extraction_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle
!pip install --upgrade kaggle

In [ ]:
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Set up Kaggle resources

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
! kaggle competitions download -c tweet-sentiment-extraction

  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 133MB/s]


In [ ]:
! unzip tweet-sentiment-extraction.zip

Archive:  tweet-sentiment-extraction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
test_data = pd.read_csv("/content/test.csv")
train_data = pd.read_csv("/content/train.csv")

test_data.head()
train_data.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


## Data analysis

In [ ]:
# Check basic information
train_data.describe()

,textID,text,selected_text,sentiment
count,27481,27480,27480,27481
unique,27481,27480,22463,3
top,cb774db0d1,"I`d have responded, if I were going",good,neutral
freq,1,1,199,11118


In [ ]:
test_data.describe()

,textID,text,sentiment
count,3534,3534,3534
unique,3534,3534,3
top,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
freq,1,1,1430


In [ ]:
# Check for null values of train data
train_data.isna().sum()

textID           0
text             1
selected_text    1
sentiment        0
dtype: int64

In [ ]:
# Drop null values
train_data.dropna(inplace=True)

In [ ]:
# Check for null values of test data
test_data.isna().sum()

textID       0
text         0
sentiment    0
dtype: int64

In [ ]:
# Drop null values
test_data.dropna(inplace=True)

In [ ]:
# Count seniment values
train_data['sentiment'].value_counts()

neutral     11117
positive     8582
negative     7781
Name: sentiment, dtype: int64

In [ ]:
# Analysis text length
train_data['text_length'] = train_data['text'].apply(len)
train_data.groupby('sentiment')['text_length'].mean()

sentiment
negative    70.488112
neutral     65.206800
positive    70.419133
Name: text_length, dtype: float64

In [ ]:
# Analysis selected text length
train_data['selected_text_length'] = train_data['selected_text'].apply(len)
train_data.groupby('sentiment')['selected_text_length'].mean()


sentiment
negative    19.970698
neutral     62.765134
positive    18.124680
Name: selected_text_length, dtype: float64

In [ ]:
# ANOVA results analysis
from scipy import stats

f_val, p_val = stats.f_oneway(train_data[train_data['sentiment'] == 'positive']['text_length'],
                              train_data[train_data['sentiment'] == 'negative']['text_length'],
                              train_data[train_data['sentiment'] == 'neutral']['text_length'])

print("ANOVA Test Results:")
print(f"F-statistic: {f_val}")
print(f"P-value: {p_val}")

# Interpret the results
alpha = 0.05
if p_val<alpha:
     print("The means of at least two groups are significantly different.")
else:
    print("There is no significant difference in the means of the groups.")

ANOVA Test Results:
F-statistic: 72.2127709711816
P-value: 5.254438748898152e-32
The means of at least two groups are significantly different.


## Data Processing

In [ ]:
import re
import string

def clean_text(text):
  text = text.lower()
  text = re.sub(r"what's", "what is ", text)
  text = re.sub(r"\'s", " ", text)
  text = re.sub(r"\'ve", " have ", text)
  text = re.sub(r"can't", "cannot ", text)
  text = re.sub(r"n't", " not ", text)
  return text

train_data['text'] = train_data['text'].apply(clean_text)
test_data['text'] = test_data['text'].apply(clean_text)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenize the text
token = Tokenizer(num_words = 500)
token.fit_on_texts(train_data['text'])

# Convert texts to sequence of integers
train_sequences = token.texts_to_sequences(train_data['text'])
test_sequences = token.texts_to_sequences(test_data['text'])

# Convert labels to categorical one-hot encoding
train_labels = pd.get_dummies(train_data['sentiment']).values
test_labels = pd.get_dummies(test_data['sentiment']).values


In [ ]:
# Vectorize the text
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = max([len(x) for x in train_sequences])
train_padded = pad_sequences(train_sequences, maxlen = max_len, padding = "post", truncating = "post")
test_padded = pad_sequences(test_sequences, maxlen = max_len, padding = "post", truncating = "post")

print(max_len)
print(train_padded.shape)
print(test_padded.shape)


32
(27480, 32)
(3534, 32)


## Build the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def lstm_model(max_len):
  model = Sequential()

  # Add embedding layer
  model.add(Embedding(input_dim = 500, output_dim = 32, input_length = max_len))

  # Add LSTM layer
  model.add(LSTM(64, return_sequences=True))
  model.add(Dropout(0.5))

  # Add LSTM layer without returning sequences
  model.add(LSTM(32))
  model.add(Dropout(0.5))

  # Add dense layer
  model.add(Dense(3, activation='sigmoid'))

  return model

def gru_model(max_len):
  model = Sequential()

  # Add embedding layer
  model.add(Embedding(input_dim = 500, output_dim = 32, input_length = max_len))
  # Add GRU layer
  model.add(GRU(128, return_sequences=True))
  model.add(Dropout(0.5))

  # Add GRU layer without returning sequences
  model.add(GRU(32))
  model.add(Dropout(0.5))

  # Add dense layer
  model.add(Dense(3, activation='sigmoid'))

  return model

In [ ]:
lstm = lstm_model(max_len)
lstm.summary()
# Complie the models
lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
gru_model(max_len).compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 32)            16000     
                                                                 
 lstm (LSTM)                 (None, 32, 64)            24832     
                                                                 
 dropout (Dropout)           (None, 32, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 3)                 99        
                                                                 
Total params: 53347 (208.39 KB)
Trainable params: 53347 

In [ ]:
gru = gru_model(max_len)
gru.summary()
# Complie the models
gru.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 32, 32)            16000     
                                                                 
 gru_2 (GRU)                 (None, 32, 128)           62208     
                                                                 
 dropout_4 (Dropout)         (None, 32, 128)           0         
                                                                 
 gru_3 (GRU)                 (None, 32)                15552     
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 93859 (366.64 KB)
Trainable params: 9385

## Train the models

In [ ]:
# Train the models with early stopping
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

In [ ]:
lstm_history = lstm.fit(train_padded, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
687/687 [==============================] - 32s 37ms/step - loss: 1.0219 - accuracy: 0.4775 - val_loss: 0.8188 - val_accuracy: 0.6527
Epoch 2/10
687/687 [==============================] - 23s 34ms/step - loss: 0.8102 - accuracy: 0.6598 - val_loss: 0.7817 - val_accuracy: 0.6652
Epoch 3/10
687/687 [==============================] - 25s 37ms/step - loss: 0.7768 - accuracy: 0.6731 - val_loss: 0.7642 - val_accuracy: 0.6678
Epoch 4/10
687/687 [==============================] - 24s 35ms/step - loss: 0.7589 - accuracy: 0.6784 - val_loss: 0.7515 - val_accuracy: 0.6765
Epoch 5/10
687/687 [==============================] - 23s 34ms/step - loss: 0.7517 - accuracy: 0.6802 - val_loss: 0.7628 - val_accuracy: 0.6752
Epoch 6/10
687/687 [==============================] - 24s 35ms/step - loss: 0.7412 - accuracy: 0.6848 - val_loss: 0.7506 - val_accuracy: 0.6809
Epoch 7/10
687/687 [==============================] - 23s 34ms/step - loss: 0.7348 - accuracy: 0.6894 - val_loss: 0.7580 - val_accuracy:

In [ ]:
gru_history= gru.fit(train_padded, train_labels, epochs=10, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/10
687/687 [==============================] - 40s 52ms/step - loss: 1.0896 - accuracy: 0.3977 - val_loss: 1.0874 - val_accuracy: 0.4127
Epoch 2/10
687/687 [==============================] - 38s 55ms/step - loss: 1.0894 - accuracy: 0.4025 - val_loss: 1.0858 - val_accuracy: 0.4127
Epoch 3/10
687/687 [==============================] - 35s 50ms/step - loss: 1.0880 - accuracy: 0.4026 - val_loss: 1.0870 - val_accuracy: 0.4127
Epoch 4/10
687/687 [==============================] - 38s 55ms/step - loss: 1.0884 - accuracy: 0.4028 - val_loss: 1.0857 - val_accuracy: 0.4127
Epoch 5/10
687/687 [==============================] - 35s 51ms/step - loss: 1.0883 - accuracy: 0.4025 - val_loss: 1.0855 - val_accuracy: 0.4127
Epoch 6/10
687/687 [==============================] - 37s 54ms/step - loss: 0.9598 - accuracy: 0.5236 - val_loss: 0.7822 - val_accuracy: 0.6658
Epoch 7/10
687/687 [==============================] - 35s 51ms/step - loss: 0.7834 - accuracy: 0.6618 - val_loss: 0.7622 - val_accuracy:

In [ ]:
def jaccard(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def evaluate_model(model, tokenizer, data, true_texts):
    """
    Evaluate the model using the Jaccard score.
    - model: The trained model (LSTM or GRU)
    - tokenizer: Tokenizer used for the model
    - data: The input data for prediction (features)
    - true_texts: The true output texts (labels)

    Returns the average Jaccard score for the dataset.
    """
    # Generate predictions
    predictions = model.predict(data)

    pred_texts = [" ".join(tokenizer.sequences_to_texts([p])) for p in predictions]

    # Compute Jaccard scores
    scores = [jaccard(pred, true) for pred, true in zip(pred_texts, true_texts)]

    # Return the average Jaccard score
    return sum(scores) / len(scores)

In [ ]:
# Evaluate on test data
test_loss, test_acc = lstm.evaluate(test_padded, test_labels)
print("Test accuracy:", test_acc)

test_loss, test_acc = gru.evaluate(test_padded, test_labels)
print("Test accuracy:", test_acc)

111/111 [==============================] - 2s 14ms/step - loss: 0.7518 - accuracy: 0.6672
Test accuracy: 0.6672325730323792
111/111 [==============================] - 2s 16ms/step - loss: 0.7480 - accuracy: 0.6692
Test accuracy: 0.6692133545875549


In [ ]:
# Model accuracy
lstm_val_acc = max(lstm_history.history['val_accuracy'])
gru_val_acc = max(gru_history.history['val_accuracy'])

print("LSTM validation accuracy:", lstm_val_acc)
print("GRU validation accuracy:", gru_val_acc)

LSTM validation accuracy: 0.6808587908744812
GRU validation accuracy: 0.677219808101654


In [ ]:
# Confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
lstm_pred = lstm.predict(test_padded)
gru_pred = gru.predict(test_padded)

print(classification_report(test_labels.argmax(axis=1), lstm_pred.argmax(axis=1)))
print(classification_report(test_labels.argmax(axis=1), gru_pred.argmax(axis=1)))

111/111 [==============================] - 2s 14ms/step
              precision    recall  f1-score   support

           0       0.78      0.46      0.58      1001
           1       0.58      0.79      0.67      1430
           2       0.76      0.70      0.73      1103

    accuracy                           0.67      3534
   macro avg       0.71      0.65      0.66      3534
weighted avg       0.70      0.67      0.66      3534

              precision    recall  f1-score   support

           0       0.75      0.47      0.58      1001
           1       0.58      0.79      0.67      1430
           2       0.79      0.69      0.74      1103

    accuracy                           0.67      3534
   macro avg       0.71      0.65      0.66      3534
weighted avg       0.69      0.67      0.67      3534



In [ ]:
from datetime import datetime

def save_model(model, prefix =''):
  # Get the current time and filename
  current_time = datetime.now().strftime("%Y-%m-%d-%H%M%S")
  filename = f"{prefix}model_{current_time}.csv"
  # Save the models
  model.save(filename)
  print(f"Model saved to {filename}")

save_model(lstm, prefix = "lstm")
save_model(gru, prefix = "gru")